In [73]:
import math

from keras.models import Model
from keras.layers import Input, Lambda
from keras.applications import xception
from keras.preprocessing import image
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score


In [31]:
base_model = xception.Xception(weights='imagenet', include_top=False, pooling='avg')

In [56]:
inputs = Input(shape=(299,299, 3))

In [57]:
x = Lambda(xception.preprocess_input)(inputs)

In [59]:
outputs = base_model(x)

In [60]:
model = Model(inputs, outputs)

In [ ]:
model.compile()

In [102]:
batch_size = 32
competition_name = 'dog-breed-identification'
data_dir = '/opt/notebooks/data/' + competition_name + '/preprocessed'
gen = image.ImageDataGenerator()
batches = gen.flow_from_directory(data_dir+'/train', shuffle=False, target_size=(299, 299), batch_size=batch_size)
batches_val = gen.flow_from_directory(data_dir+'/valid', shuffle=False, target_size=(299, 299), batch_size=batch_size)
y_encode = batches.classes
y_val_encode = batches_val.classes

Found 8222 images belonging to 120 classes.
Found 2000 images belonging to 120 classes.


In [103]:
nb_batches = math.ceil(batches.n/batch_size)
bf_x=model.predict_generator(batches, steps=nb_batches, verbose=1)

257/257 [==============================] - 124s   


In [104]:
nb_batches_val = math.ceil(batches_val.n/batch_size)
bf_val_x=model.predict_generator(batches_val, steps=nb_batches_val, verbose=1)

63/63 [==============================] - 30s    


In [105]:
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs')
logreg.fit(bf_x, y_encode)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [106]:
valid_probs = logreg.predict_proba(bf_val_x)
valid_preds = logreg.predict(bf_val_x)

In [108]:
log_loss(y_val_encode, valid_probs)

0.32573993342313973

In [109]:
accuracy_score(y_val_encode, valid_preds)

0.90400000000000003

In [32]:
inputs = Input(shape=(224, 224, 3))

In [33]:
inputs

<tf.Tensor 'input_13:0' shape=(?, 224, 224, 3) dtype=float32>

In [16]:
y = base_model(inputs)

In [17]:
model = Model(inputs=inputs, outputs=y)

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Model)                (None, 1000)              138357544 
Total params: 138,357,544
Trainable params: 138,357,544
Non-trainable params: 0
_________________________________________________________________


In [21]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [2]:
vgg16.preprocess_input

<function keras.applications.imagenet_utils.preprocess_input>

In [ ]:
vgg_bottleneck = vgg16.VGG16(weights='imagenet', include_top=False, pooling='avg')

In [3]:
inputs = Input()
Lambda(vgg16.preprocess_input, 
       output_shape=(3, 224, 224)
      )